# Doing some Imports

In [2]:
import numpy as np
import pickle
import tensorflow as tf
import keras_tuner as kt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras_tuner import RandomSearch
from tensorflow.keras.optimizers import Adam
# Import necessary items from Keras
from keras.models import Sequential
from keras.layers import Activation, Dropout, UpSampling2D
from keras.layers import Conv2DTranspose, Conv2D, MaxPooling2D
#from keras.layers.normalization import BatchNormalization
from tensorflow.keras.layers import (
    BatchNormalization, SeparableConv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense
)
from keras.preprocessing.image import ImageDataGenerator
from keras import regularizers

## Create your model

In [3]:
def create_model(hp):
    # Create the actual neural network here
    hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 
    model = Sequential()
    # Normalizes incoming inputs. First layer needs the input shape to work
    model.add(BatchNormalization(input_shape= X_train.shape[1:]))

    # Below layers were re-named for easier reading of model summary; this not necessary
    # Conv+Relu Layer 1
    model.add(Conv2D(8, (3, 3), padding='same', strides=(1,1), activation = 'relu'))
    # Conv+Relu Layer 2
    model.add(Conv2D(8, (3, 3), padding='same', strides=(1,1), activation = 'relu'))
    
     # Pooling 1
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Conv+Relu Layer 3
    model.add(Conv2D(16, (3, 3), padding='same', strides=(1,1), activation = 'relu')) 
    model.add(Dropout(0.2))

    # Conv+Relu Layer 4
    model.add(Conv2D(32, (3, 3), padding='same', strides=(1,1), activation = 'relu'))
    model.add(Dropout(0.2))

    # Conv+Relu Layer 5
    model.add(Conv2D(32, (3, 3), padding='same', strides=(1,1), activation = 'relu'))
    model.add(Dropout(0.2))

    # Pooling 2
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Conv Layer 6
    model.add(Conv2D(64, (3, 3), padding='same', strides=(1,1), activation = 'relu'))
    model.add(Dropout(0.2))

    # Conv Layer 7
    model.add(Conv2D(64, (3, 3), padding='same', strides=(1,1), activation = 'relu'))
    model.add(Dropout(0.2))

    # Pooling 3
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Upsample 1
    model.add(UpSampling2D(size=(2, 2)))

    # Deconv+Relu 1
    model.add(Conv2DTranspose(64, (3, 3), padding='same', strides=(1,1), activation = 'relu'))
    model.add(Dropout(0.2))

    # Deconv+Relu 2
    model.add(Conv2DTranspose(64, (3, 3), padding='same', strides=(1,1), activation = 'relu'))
    model.add(Dropout(0.2))

    # Upsample 2
    model.add(UpSampling2D(size=(2, 2)))

    # Deconv+Relu 3
    model.add(Conv2DTranspose(32, (3, 3), padding='same', strides=(1,1), activation = 'relu'))
    model.add(Dropout(0.2))

    # Deconv+Relu 4
    model.add(Conv2DTranspose(32, (3, 3), padding='same', strides=(1,1), activation = 'relu'))
    model.add(Dropout(0.2))

    # Deconv+Relu 5
    model.add(Conv2DTranspose(16, (3, 3), padding='same', strides=(1,1), activation = 'relu'))
    model.add(Dropout(0.2))

    # Upsample 3
    model.add(UpSampling2D(size=(2, 2)))

    # Deconv+Relu 6
    model.add(Conv2DTranspose(16, (3, 3), padding='same', strides=(1,1), activation = 'relu'))

    # Final layer - only including one channel so 1 filter
    model.add(Conv2DTranspose(1, (3, 3), padding='same', strides=(1,1), activation = 'relu'))
    model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=hp_learning_rate), metrics=['accuracy'])
   
    return model

## Load the data

In [4]:
# Load training images
train_images = pickle.load(open("full_CNN_train.p", "rb" ))

# Load image labels
labels = pickle.load(open("full_CNN_labels.p", "rb" ))

# Make into arrays as the neural network wants these
train_images = np.array(train_images)
labels = np.array(labels)


## Preprocess the data

In [5]:
 # Normalize labels - training images get normalized to start in the network
labels = labels / 255


## Create the train and test splits

In [6]:
# Shuffle images along with their labels, then split into training/validation sets
train_images, labels = shuffle(train_images, labels)
# Test size may be 10% or 20%
X_train, X_val, y_train, y_val = train_test_split(train_images, labels, test_size=0.1)


In [12]:
tuner = kt.RandomSearch(create_model,
                        objective = 'val_accuracy', 
                        max_trials = 1,
                        overwrite=True)
    

In [13]:
# Batch size, epochs and pool size below are all paramaters to fiddle with for optimization
batch_size = 64
epochs = 10
#pool_size = (2, 2)
#input_shape = X_train.shape[1:]

In [14]:
# Create the neural network
#model = create_model
datagen = ImageDataGenerator()
datagen.fit(X_train)


In [ ]:
tuner.search(X_train, y_train, epochs = epochs,batch_size =batch_size , validation_data = (X_val, y_val))

## Train your model

In [51]:
# TODO: Train your model

# Compiling and training the model
#model.compile(optimizer='Adam', loss='mean_squared_error')
model.fit(datagen.flow(X_train, y_train, batch_size=batch_size), steps_per_epoch=len(X_train)/batch_size,
#epochs=epochs, verbose=1, validation_data=(X_val, y_val))

    

SyntaxError: unexpected EOF while parsing (<ipython-input-51-8c9cb5e9eec2>, line 8)

In [55]:
# Freeze layers since training is done
#tuner.trainable = False
#model.compile(optimizer='Adam', loss='mean_squared_error')
best_model = tuner.get_best_models(1)[0]
# What were the best hyperparameters?
best_hyperparameters = tuner.get_best_hyperparameters(1)[0] 


## Save your model

In [57]:
# Save model architecture and weights
best_model.save('full_CNN_model.h5')

# Show summary of model
best_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 80, 160, 3)        12        
_________________________________________________________________
conv2d (Conv2D)              (None, 80, 160, 8)        224       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 80, 160, 8)        584       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 40, 80, 8)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 40, 80, 16)        1168      
_________________________________________________________________
dropout (Dropout)            (None, 40, 80, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 40, 80, 32)        4